In [1]:
%load_ext nb_black

<IPython.core.display.Javascript object>

## Import Dependencies

In [2]:
# misc
import math
import time
from pathlib import Path

# scientific
import numpy as np
import beatbrain
from beatbrain import utils

# visualization
from IPython import display
import seaborn as sns
import matplotlib.pyplot as plt

<IPython.core.display.Javascript object>

In [3]:
# Tensorflow
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import Model, Input
from tensorflow.keras import backend
from tensorflow.keras import optimizers

from tensorflow.keras.layers import Conv2D, Conv2DTranspose, Dense, Lambda, Reshape
from tensorflow.keras.callbacks import (
    TensorBoard,
    ReduceLROnPlateau,
    EarlyStopping,
    ModelCheckpoint,
    TerminateOnNaN,
)

<IPython.core.display.Javascript object>

In [4]:
sns.set()
sns.set_style("white")
%matplotlib inline

<IPython.core.display.Javascript object>

## Load Datasets

In [5]:
# Input
DATA_ROOT = Path("../data/fma/image")
IMAGE_DIMS = [512, 640, 1]
BATCH_SIZE = 4

<IPython.core.display.Javascript object>

In [6]:
train_dataset = utils.load_dataset(
    DATA_ROOT / "train", batch_size=BATCH_SIZE, parallel=False
)
test_dataset = utils.load_dataset(
    DATA_ROOT / "test", batch_size=BATCH_SIZE, parallel=False
)

Checking input type(s) in '/media/data/beatbrain_data/fma/image/train'...
Determined input type to be 'IMAGE'
Checking input type(s) in '/media/data/beatbrain_data/fma/image/test'...
Determined input type to be 'IMAGE'


<IPython.core.display.Javascript object>

## Define Model Architecture

In [7]:
def build_cvae(latent_dim, input_shape, num_conv=3, batch_size=1, learning_rate=1e-3):
    def reparam(args):
        z_mean, z_log_var = args
        dim = tf.keras.backend.int_shape(z_mean)[1]
        eps = tf.keras.backend.random_normal(shape=(batch_size, dim))
        return eps * tf.exp(z_log_var * 0.5) + z_mean

    encoder_input = tf.keras.Input(shape=input_shape, batch_size=batch_size)
    e = tf.keras.layers.Conv2D(
        filters=32, kernel_size=3, strides=(2, 2), activation="relu"
    )(encoder_input)
    for i in range(num_conv):
        e = tf.keras.layers.Conv2D(
            filters=64, kernel_size=3, strides=(2, 2), activation="relu"
        )(e)
    decoder_input_shape = tf.keras.backend.int_shape(e)
    # decoder_input_shape: (1, 31, 39, 64)
    e = tf.keras.layers.Flatten()(e)
    e = tf.keras.layers.Dense(16)(e)
    z_mean = tf.keras.layers.Dense(latent_dim)(e)
    z_log_var = tf.keras.layers.Dense(latent_dim)(e)
    z = tf.keras.layers.Lambda(reparam, output_shape=(latent_dim,))([z_mean, z_log_var])
    encoder = tf.keras.Model(encoder_input, [z_mean, z_log_var, z], name="encoder")

    decoder_input_shape = (
        1,
        input_shape[0] // (2 ** (num_conv + 1)),
        input_shape[1] // (2 ** (num_conv + 1)),
        64,
    )
    # (decoder_input_shape): (32, 40, 64)

    decoder_input = tf.keras.Input(shape=(latent_dim,))
    d = tf.keras.layers.Dense(
        decoder_input_shape[1] * decoder_input_shape[2] * decoder_input_shape[3],
        activation=tf.nn.relu,
    )(decoder_input)
    d = tf.keras.layers.Reshape(
        target_shape=(
            decoder_input_shape[1],
            decoder_input_shape[2],
            decoder_input_shape[3],
        )
    )(d)
    # d.shape: (None, 31, 39, 64)
    for i in range(num_conv):
        d = tf.keras.layers.Conv2DTranspose(
            filters=64,
            kernel_size=3,
            strides=(2, 2),
            padding="SAME",
            activation="relu",
        )(d)
    d = tf.keras.layers.Conv2DTranspose(
        filters=32, kernel_size=3, strides=(2, 2), padding="SAME", activation="relu",
    )(d)
    decoder_output = tf.keras.layers.Conv2DTranspose(
        filters=1, kernel_size=3, strides=(1, 1), padding="SAME",
    )(d)

    decoder = tf.keras.Model(decoder_input, decoder_output, name="decoder")
    outputs = decoder(encoder(encoder_input)[2])
    model = tf.keras.Model(encoder_input, outputs, name="vae")

    assert encoder_input.shape == outputs.shape
    reconstruction_loss = tf.losses.mse(encoder_input, outputs)
    reconstruction_loss = tf.reduce_sum(reconstruction_loss, axis=[1, 2])
    log2pi = tf.math.log(2.0 * np.pi)
    logpz = log_normal_pdf(z, 0.0, 0.0)
    logqz_x = log_normal_pdf(z, z_mean, z_log_var)
    kl_loss = logqz_x - logpz
    vae_loss = tf.reduce_mean(reconstruction_loss + kl_loss)

    model.add_loss(vae_loss)
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate))
    return model, encoder, decoder


def log_normal_pdf(sample, mean, logvar, raxis=1):
    log2pi = tf.math.log(2.0 * np.pi)
    return tf.reduce_sum(
        -0.5 * ((sample - mean) ** 2.0 * tf.exp(-logvar) + logvar + log2pi), axis=raxis
    )


@tf.function
def sample(latent_dim, decoder, eps=None):
    if eps is None:
        eps = tf.random.normal(shape=(100, latent_dim))
    return decode(decoder, eps, apply_sigmoid=True)


def encode(encoder, x):
    inference = encoder(x)
    mean, logvar = tf.split(inference, num_or_size_splits=2, axis=1)
    return mean, logvar


def reparameterize(mean, logvar):
    eps = tf.random.normal(shape=mean.shape)
    return eps * tf.exp(logvar * 0.5) + mean


def decode(decoder, z, apply_sigmoid=False):
    logits = decoder(z)
    if apply_sigmoid:
        probs = tf.sigmoid(logits)
        return probs
    return logits

<IPython.core.display.Javascript object>

## Hyperparameters, Model Output, and Logging

In [13]:
# Hyperparameters
LATENT_DIM = 256
EPOCHS = 50
NUM_CONV = 3
LEARNING_RATE = 1e-3

# Callback options
EARLY_STOP_PATIENCE = 3
REDUCE_LR_PATIENCE = 1

# Outputs
MODEL_NAME = "2d-CVAE"
MODEL_DIR = Path("../models")
MODEL_DIR.mkdir(exist_ok=True, parents=True)
TENSORBOARD_DIR = Path("../logs")

<IPython.core.display.Javascript object>

## Instantiate Model

In [20]:
model, encoder, decoder = build_cvae(
    LATENT_DIM,
    IMAGE_DIMS,
    num_conv=NUM_CONV,
    batch_size=BATCH_SIZE,
    learning_rate=LEARNING_RATE,
)
model.summary()

Model: "vae"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_9 (InputLayer)            [(4, 512, 640, 1)]   0                                            
__________________________________________________________________________________________________
encoder (Model)                 [(4, 256), (4, 256), 1339408     input_9[0][0]                    
__________________________________________________________________________________________________
decoder (Model)                 multiple             21182977    encoder[1][2]                    
__________________________________________________________________________________________________
conv2d_16 (Conv2D)              (4, 255, 319, 32)    320         input_9[0][0]                    
________________________________________________________________________________________________

<IPython.core.display.Javascript object>

## Train Model

In [22]:
# Keras training callbacks
tensorboard = TensorBoard(
    log_dir=TENSORBOARD_DIR / MODEL_NAME, update_freq=256, profile_batch=0,
)
reduce_lr = ReduceLROnPlateau(patience=1, factor=0.1, min_lr=1e-5, verbose=1,)
early_stop = EarlyStopping(patience=3, verbose=1,)
model_saver = ModelCheckpoint(
    f"{MODEL_DIR / MODEL_NAME}", save_best_only=True, verbose=1,
)

# Train model
model.fit_generator(
    train_dataset.take(100),
    epochs=EPOCHS,
    callbacks=[tensorboard, reduce_lr, early_stop, model_saver],
    validation_data=test_dataset.take(5),
)

Epoch 1/50
 99/100 [============================>.] - ETA: 0s - loss: 34513.1446
Epoch 00001: val_loss improved from inf to 11702.08906, saving model to ../models/2d-CVAE
INFO:tensorflow:Assets written to: ../models/2d-CVAE/assets
100/100 [==============================] - 20s 196ms/step - loss: 34306.3847 - val_loss: 11702.0891
Epoch 2/50
 99/100 [============================>.] - ETA: 0s - loss: 7983.9983
Epoch 00002: val_loss improved from 11702.08906 to 7636.48271, saving model to ../models/2d-CVAE
INFO:tensorflow:Assets written to: ../models/2d-CVAE/assets
100/100 [==============================] - 20s 201ms/step - loss: 7969.8519 - val_loss: 7636.4827
Epoch 3/50
 99/100 [============================>.] - ETA: 0s - loss: 6015.5340
Epoch 00003: val_loss improved from 7636.48271 to 5120.87354, saving model to ../models/2d-CVAE
INFO:tensorflow:Assets written to: ../models/2d-CVAE/assets
100/100 [==============================] - 20s 203ms/step - loss: 6001.1632 - val_loss: 5120.8735


KeyboardInterrupt: 

<IPython.core.display.Javascript object>

In [ ]:
loaded_model = tf.keras.models.load_model(MODEL_PATH / f"{MODEL_NAME}.h5")
loaded

In [ ]:
[l.name for l in loaded.get_layer("Encoder").layers]

In [ ]:
EXAMPLES_TO_GENERATE = 16
INTERPOLATION_POINTS = 9
OUTPUT_DIR = Path("../data/output/images")

# keeping the random vector constant for generation (prediction) so
# it will be easier to see the improvement.
random_vector_for_generation = tf.random.normal(
    shape=[EXAMPLES_TO_GENERATE, LATENT_DIM]
)

In [ ]:
sample = next(train_dataset.take(1))
beatbrain.display.show_spec(utils.denormalize_spectrogram(sample[0, ..., 0]), title="Orginal")
plt.show()
sns.distplot(train_dataset.flatten())
plt.show()

In [ ]:
prediction = model(train_dataset)
beatbrain.display.show_spec(utils.denormalize_spectrogram(model(sample)[0, ..., 0].numpy()), title="Predicted")
plt.show()
sns.distplot(model(train_dataset).numpy().flatten())

In [ ]:
N_FFT = 4096
HOP_LENGTH = 256
SAMPLE_RATE = 32768

In [ ]:
sample_recon = utils.spectrogram_to_audio(sample[0, ..., 0], denormalize=True, n_fft=N_FFT, hop_length=HOP_LENGTH, sr=SAMPLE_RATE)

In [ ]:
display.Audio(sample_recon, rate=SAMPLE_RATE)

In [ ]:
prediction_recon = utils.spectrogram_to_audio(prediction[0, ..., 0], denormalize=True, n_fft=N_FFT, hop_length=HOP_LENGTH, sr=SAMPLE_RATE)

In [ ]:
display.Audio(prediction_recon, rate=SAMPLE_RATE)

In [ ]:
def generate_and_save_images(decoder, epoch, test_input):
    num_plots = math.ceil(math.sqrt(len(test_input)))
    predictions = sample(LATENT_DIM, decoder, eps=test_input)
    fig = plt.figure(figsize=(12, 12))
    fig.subplots_adjust(hspace=0, wspace=0)
    for i, pred in enumerate(predictions):
        plt.subplot(num_plots, num_plots, i + 1)
        plt.imshow(pred[:, :, 0], cmap='gray', vmin=0, vmax=1)
        plt.axis('off')
        output_dir = os.path.join(OUTPUT_DIR, 'progress', str(i))
        os.makedirs(output_dir, exist_ok=True)
        image = Image.fromarray(pred[:, :, 0].numpy(), mode='F')
#         image.save(os.path.join(output_dir, f"epoch_{epoch}.tiff"))
        image.save(os.path.join(output_dir, f"spec.tiff"))
    plt.show()

In [ ]:
from time import time
import librosa

SR = 32768
N_FFT = 4096
N_MELS = 512
HOP_LENGTH = 256
DURATION = 5
A = "../data/fma/audio/000002.mp3"
B = "../data/fma/audio/000005.mp3"

interp_dir = Path(f"interpolation/{int(time())}")
interp_dir.mkdir(exist_ok=True, parents=True)

x, _ = librosa.load(A, sr=SR, duration=DURATION)
y, _ = librosa.load(B, sr=SR, duration=DURATION)
x = librosa.feature.melspectrogram(x, sr=SR, n_fft=N_FFT, hop_length=HOP_LENGTH, n_mels=N_MELS)
y = librosa.feature.melspectrogram(y, sr=SR, n_fft=N_FFT, hop_length=HOP_LENGTH, n_mels=N_MELS)
x, y = x[0, None], y[None, :, None]
print(x.shape)
x_mean, x_logvar = model.encode(x)
y_mean, y_logvar = model.encode(y)

# Reconstruction
fig = plt.figure(figsize=(12, 6))
plt.subplot(121)
plt.imshow(x[0, ..., 0], cmap='gray', vmin=0, vmax=1)
plt.axis('off')
plt.subplot(122)
x_recon = model.sample(model.reparameterize(x_mean, x_logvar))
plt.imshow(x_recon[0, ..., 0], cmap='gray', vmin=0, vmax=1)
plt.axis('off')
plt.show()

# Interpolation
fractions = np.linspace(0, 1, num=INTERPOLATION_POINTS)[:, None]
means = (x_mean * (1 - fractions)) + (y_mean * fractions)  # Interpolated latent vectors
logvars = (x_logvar * (1 - fractions)) + (y_logvar * fractions)  # Interpolated latent vectors
points = model.reparameterize(means, logvars)
interpolated = model.sample(points)

num_plots = math.ceil(math.sqrt(INTERPOLATION_POINTS))
fig = plt.figure(figsize=(12, 12))
fig.subplots_adjust(hspace=0, wspace=0.03)
for i, pred in enumerate(interpolated):
    Image.fromarray(pred[:, :, 0].numpy(), mode='F').save(interp_dir.joinpath(f'{i}.tiff'))
    plt.subplot(num_plots, num_plots, i + 1)
    if i == 0:
        Image.fromarray(x[0, :, :, 0].numpy(), mode='F').save(interp_dir.joinpath(f'a.tiff'))
        plt.imshow(x[0, ..., 0], cmap='gray', vmin=0, vmax=1)
    elif i == INTERPOLATION_POINTS - 1:
        Image.fromarray(y[0, :, :, 0].numpy(), mode='F').save(interp_dir.joinpath(f'b.tiff'))
        plt.imshow(y[0, ..., 0], cmap='gray', vmin=0, vmax=1)
    else:
        plt.imshow(pred[:, :, 0], cmap='gray', vmin=0, vmax=1)
    plt.axis('off')

plt.savefig(interp_dir.joinpath('interpolation.png'), bbox_inches='tight')